<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/inspect_train_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determine confidence threshold for Image Type Classification Models
---
*Last Updated 2 December 2024*   
Choose which trained model and confidence threshold values to use for classifying EOL images as maps, phylogenies, illustrations, or herbarium sheets. Threshold values should be chosen that maximize coverage and minimize error.

First, choose the 2 best models trained in [image_type_train.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/image_type_preprocessing.ipynb). Then, run this notebook.

Run 500 images per class (map, phylogeny, illustration, herbarium sheet) through the best models chosen in image_type_train.ipynb for validation of model performance. Plot a confusion matrix, generate a classification report, and plot histograms of true and false predictions per class at binned confidence intervals to find the best performance by class and confidence threshold. (This is helpful because all models may not learn classes equally well).

***Models were trained in Python 2 and TF 1 in October 2020: MobileNet SSD v2 (Training Session 13) was trained for 3 hours to 30 epochs with Batch Size=16, Lr=0.00001, Dropout=0.3, epsilon=1e-7, Adam optimizer. Final validation accuracy = 0.90. Inception v3 (Training Session 11) was trained for 3.5 hours to 30 epochs with Batch Size=16, Lr=0.0001, Dropout=0.2, epsilon=1, Adam optimizer. Final validation accuracy = 0.89.***

Notes:   
* Run code blocks by pressing play button in brackets on left
* Before you you start: change the runtime to "GPU" with "High RAM"
* Change parameters using form fields on right (find details at corresponding lines of code by searching '#@param')

## Installs & Imports
---

In [ ]:
#@title Choose where to save results (or keep defaults) & set up environment
import os

# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]
print("Saving results ", save)

# Mount google drive to export file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type of classification pipeline
classif_type = "image_type" #@param ["image_type", "rating"] {allow-input: true}

# Type in the path to your working directory in form field to right
basewd = "/content/drive/MyDrive/train/tf2" #@param ["/content/drive/MyDrive/train/tf2"] {allow-input: true}
basewd = basewd + '/' + classif_type

# Folder where inspect results outputs will be saved
folder = "inspect_resul" # @param ["inspect_resul","results"] {"allow-input":true}
cwd = basewd + '/' + folder
print("\nWorking directory set to: \n", cwd)

# Enter image classes of interest in form field
filters = ['herb', 'illus', 'map', 'null', 'phylo'] # @param ["['herb', 'illus', 'map', 'null', 'phylo']"] {"type":"raw","allow-input":true}

# Download helper_funcs folder
!pip3 -q install --upgrade gdown
!gdown  1TfpSLwbt6i0OMdbbjcTPVFBP6vYc3CY_
!tar -xzvf helper_funcs.tar.gz -C .

# Install requirements.txt
!pip3 -q install -r requirements.txt

In [ ]:
#@title Choose saved model parameters (if using EOL model, defaults are already selected)
from setup import setup_dirs, load_saved_model, get_model_info, unpack_EOL_model
%cd /content

# Use EOL pre-trained model for object detection?
use_EOL_model = True #@param {type: "boolean"}

# Download EOL models, if appropriate
if (use_EOL_model==True) & (os.path.exists("11.zip")==False):
    !gdown 1Sxp742kescTGAUKlVkRR2hcoVo39y4pd
if (use_EOL_model==True) & (os.path.exists("13.zip")==False):
    !gdown  1zpC23yiYUXZmVn7q6xhnhBTH9wTBK5Tf

# If using your own trained model, change values to match your trained model
dataset_labels = filters
saved_models_folder = "saved_models" #@param ["train/saved_models/"] {allow-input: true}
saved_models_dir = basewd + '/' + saved_models_folder + '/'
print("\nSaved models directory set to: \n", saved_models_dir)
TRAIN_SESS_NUMS = ["11", "13"] # @param ["[\"11\", \"13\"]"] {"type":"raw","allow-input":true}

# Set up directory structure
setup_dirs(cwd)
%cd $basewd

# Unpack EOL saved models
for TRAIN_SESS_NUM in TRAIN_SESS_NUMS:
    trained_model_dir = saved_models_dir + TRAIN_SESS_NUM + '/'
    unpack_EOL_model(use_EOL_model, trained_model_dir, basewd, TRAIN_SESS_NUM, classif_type)

# Folder where image metadata was saved in rating_preprocessing.ipynb
data_folder = "pre-processing/image_data" #@param ["pre-processing/image_data"] {allow-input: true}
data_wd = basewd + '/' + data_folder
if not os.path.exists(data_wd):
    os.makedirs(data_wd)
    print("\nDownload image bundles for {} classes {}...\n".format(classif_type, filters))
    %cd $data_wd
    file_ids = ['1Bkh2-TZSIKCCoKOTNr2L65BwR92Nx0vZ', '1m2sOLpUOWsw5RwzRtvj0mqH8aPloqnE_', \
                '1EIwPxyrawXnTPMyvO8f4nc1e3HALrTp9', '16I-_Qbh2IX_1Oz5wqlE6uzWXB2VhjE3e', \
                '1hQNgRLZWZu77XAxBwQIJOgRmWCCcpMos']
    for file_id in file_ids:
        !gdown $file_id
print("\nImage metadata directory set to: \n", data_wd)

In [ ]:
#@title Import libraries

# For working with data
import itertools
import os
import numpy as np
import pandas as pd

# Suppress pandas setting with copy warning
pd.options.mode.chained_assignment = None  # default='warn'

# For downloading and displaying images
import matplotlib.pyplot as plt
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline

# For measuring inference time
import time

# For image classification and training
import tensorflow as tf
print('\n\n\nUsing Tensorflow Version: ', tf.__version__)

# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

## Run images through for classification and validating predictions (Run 1x for each trained model)   
---
Selected models from image_type_train.ipynb   
* Run 11: Inception v3
* Run 13: Mobilenet SSD v2

In [ ]:
# Define functions
from wrangle_data import read_datafile, set_start_stop, image_from_url, get_predict_info

# Make a dictionary of image type classes and corresponding image bundles
demo_dict = {'herb': 'herbarium_sheets_download.txt', 'illus': ['Botanical_illustrations_download.txt', 'Zoological_illustrations_download.txt'], 'map': 'maps.txt', 'null': None, 'phylo': 'Phylogeny_images.txt'}

# Set filename for saving classification results
def get_test_images(imclass):
    impath = cwd + '/pre-processing/images/' + imclass
    # If already custom-trained model, pull test images to inspect results for
    if os.path.exists(impath):
        demo = False # Not running in demo mode
        fns = os.listdir(impath)
        TEST_IMAGE_PATHS = [os.path.join(impath, fn) for fn in fns]
        print("\nUsing test images from: \n", impath)
    # If running this script to test functionality, download dummy dataset from EOL image bundles
    else:
        demo = True # Running in demo mode using only Colab Runtime files
        TEST_IMAGE_PATHS = []
        try:
            if 'illus' in imclass:
                fpath = data_wd + '/' + demo_dict[imclass][0]
            elif 'null' in imclass:
                print("\n\033[91m Null image class doesn't have demo image bundle available. Moving onto the next class.\033[0m\n")
                pass
            else:
                fpath = data_wd + '/' + demo_dict[imclass]
            df = pd.read_table(fpath)
            start = 1
            stop = start + 500
            TEST_IMAGE_PATHS = df.iloc[start:stop, 0].values.tolist()
            print("\nUsing up to 5 random images from EOL image type bundle: \n", fpath)

        except:
            pass

    return TEST_IMAGE_PATHS, demo

# Set filename for saving classification results
def set_outfpath(imclass):
    outfpath = cwd + '/' + classif_type + '_' + TRAIN_SESS_NUM + '_' + imclass + '.csv'
    print("\nSaving results to: \n", outfpath)

    return outfpath

# Load in image from file
def image_from_file(im_path):
    imga = Image.open(im_path) # rgba (with transp)
    colormode = imga.getbands()
    img = imga.convert('RGB') # convert to rgb
    image = img.resize([pixels,pixels])
    image = np.reshape(image,[1,pixels,pixels,3])
    image = image*1./255 # normalize colorspace

    return image, colormode

# Make placeholder lists to fill for each class
def make_placeholders():
    filenames = []
    confidences = []
    true_imclasses = []
    det_imclasses = []
    colormodes = []

    return filenames, confidences, true_imclasses, det_imclasses, colormodes

# Add values for each image to placeholder list
def record_results(fn, conf, true_imclass, det_imclass, colormode):
    filenames.append(fn)
    confidences.append(conf)
    true_imclasses.append(true_imclass)
    det_imclasses.append(det_imclass)
    colormodes.append(colormode)
    results = [filenames, confidences, true_imclasses, det_imclasses, colormodes]
    # Display progress message after each image
    print("Completed for {} of {} files".format(len(filenames), cutoff))

    return results

# Export results
def export_results(results, outfpath):
    results = pd.DataFrame(results)
    results = results.transpose()
    results.to_csv(outfpath, index=False, header=("filename", "confidence",
                                                     "true_id", "det_id", "colormode"))
    print("\nClassification predictions for image class {} being saved to : \n{}\n".format(
          true_imclass, outfpath))

In [ ]:
#@title Run inference for chosen Training Session Number (11, 13) and dataset size
%cd $cwd

# Choose training attempt number to inspect results for
TRAIN_SESS_NUM = "11" #@param ["11", "13"] {allow-input: true}

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for 500 images"]
print("Run: ", run)

# Load saved model
module_selection, dataset_labels = get_model_info(TRAIN_SESS_NUM)
print("Loading saved model for Train Attempt {}: {}".format(TRAIN_SESS_NUM, module_selection))
model, pixels, handle_base = load_saved_model(saved_models_dir, TRAIN_SESS_NUM, module_selection)

# Run inference for each image class to compare known versus predicted image types
true_imclasses = filters
for true_imclass in true_imclasses:
    print("Runing inference for class: {}\n".format(true_imclass))
    # Set filename for saving classification results
    outfpath = set_outfpath(true_imclass)
    # Make placeholder lists to record values for each image
    filenames, confidences, true_imclasses, det_imclasses, colormodes = make_placeholders()
    # Get test images for running inference
    df, demo = get_test_images(true_imclass)

    # Run 500 random EOL bundle images through trained model
    try:
        start, stop, cutoff = set_start_stop(run, df)
        for i, row in enumerate(df[start:stop], start=1):
            # Read in image from file
            if demo:
                url = row
                fn = str(i) + '.jpg'
                img, _, colormode = image_from_url(url, fn, pixels)
            else:
                im_path = row
                img, colormode = image_from_file(row)

            # Image classification
            start_time = time.time() # Record inference time
            predictions = model.predict(img, batch_size=1)
            label_num, conf, det_imclass = get_predict_info(predictions, row, i, stop, start, dataset_labels)
            end_time = time.time()
            print("Inference time: {} sec".format(format(end_time-start_time, '.2f')))

            # Record results in placeholder lists to inspect results in next step
            results = record_results(row, conf, str(true_imclass), str(det_imclass), colormode)

            # Set cutoff for # of predictions per class (workaround to get same # of predictions per class, even with many broken URLs)
            print("\033[92m Completed for {} of {} files \033[0m".format(len(results[0]), cutoff))
            if len(results[0])>=cutoff:
                break

    except:
        pass

    # Combine to df and export results
    export_results(results, outfpath)

print("\n\n~~~\n\033[92m Inference complete!\033[0m\n~~~")

In [ ]:
#@title Combine model outputs for image type classes

# Combine prediction files created in codeblock above
base = classif_type + '_' + TRAIN_SESS_NUM + '_'
imclasses = filters
all_filenames = [base + imclass + '.csv' for imclass in imclasses]
all_predictions = pd.concat([pd.read_csv(f, sep=',', header=0, na_filter = False) for f in all_filenames])
outfpath = base + handle_base + '_all_predictions' + '.csv'
print("Combining model prediction .csv's for all classes...\n", all_filenames, "\n")
all_predictions.to_csv(outfpath, index=False)
print("Combined results saved to: ", outfpath, "\n")
print("No. Images: {}\n".format(len(all_predictions)))
print("Model predictions for Training Attempt {}, {} with numeric classes:\n{}".format(\
      TRAIN_SESS_NUM, handle_base, all_predictions[['filename', 'true_id', 'det_id']].head()))

## Plot prediction error and confidence for each class (Run 1x for each trained model)
---   
Make a confusion matrix, classification report, and histograms. Then, use them to find a confidence threshold value for all or some classes to optimize dataset coverage and accuracy.

If you already ran images through all models for predictions and want to play with plotting, specify TRAIN_SESS_NUM for your model of interest below.

In [ ]:
#@title Load model predictions (Optional: If you want to load in all_predictions.csv from a previous run, specify TRAIN_SESS_NUM)
from setup import get_model_info

%cd $cwd
# Load combined prediction results from above
run_from_file = "no" # @param ["no","yes"]

# Copy dataframe from above
if run_from_file=="no":
    print("Using runtime data from all_predictions: ", all_predictions.head())

# Load all_predictions.csv from file for provided training attempt
else:
    # Choose training attempt number to inspect results for
    TRAIN_SESS_NUM = "11" # @param ["11","13"] {"allow-input":true}
    # Get model info
    module_selection, dataset_labels = get_model_info(TRAIN_SESS_NUM)
    handle_base, pixels = module_selection
    # Load in all_predictions from file
    base = classif_type + '_' + TRAIN_SESS_NUM + '_'
    fname = base + handle_base + '_all_predictions' + '.csv'
    all_predictions = pd.read_csv(fname, sep=',', header=0, na_filter = False)
    print("Loading prediction data from file for train attempt {}, {} with labels {} for confusion matrix and histograms: \n{}".format(TRAIN_SESS_NUM, handle_base, dataset_labels, all_predictions[['filename', 'true_id', 'det_id']].head()))

In [ ]:
# Define functions

# Plot confusion matrix
# Modified from http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html on 22 Oct 23
def plot_confusion_matrix(cm, target_names, title='Confusion matrix',
                          cmap=None, normalize=True):
    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy
    # Set colormap
    if cmap is None:
        cmap = plt.get_cmap('Blues')
    # Build figure
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    # Normalize data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    # Add labels
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.2f}; misclass={:0.2f}'.format(accuracy, misclass))
    plt.show()

    return fig

# Calculate prediction accuracy
def get_accuracy(obs, all_vals):
    # obs = observed, all_vals = observed + expected
    if obs:
        accuracy = format((obs/all_vals), '.2f')
    else:
        accuracy = 0

    return accuracy

# Valide predictions by image class (and optionally, by: taxon)
def validate_predict(df, inspect_by_taxon, taxon):
    # If inspecting for taxon-specific images only
    if inspect_by_taxon:
        taxon = taxon
        df = df.loc[df.ancestry.str.contains(taxon, case=False, na=False)]
        print("Inspecting results for {}:\n{}".format(taxon, df.head()))

    # Validate predictions
    # Check where true image types and model-determined classes match
    df['det'] = (df['true_id'] == df['det_id'])
    tru = df.loc[df.det, :] # True ID
    fal = df.loc[~df.det, :] # False ID

    return tru, fal, taxon

# Plot results by image class
def plot_predict_x_conf(tru, fal, thresh, imclasses=imclasses):
    # Break up predictions by image class and confidence values
    # Define variables
    c0,c1,c2,c3,c4 = [imclasses[i] for i in range(0, len(imclasses))]

    # Check how many true/false predictions are at each confidence value
    # Class 0 - 'herb'
    c0t = tru.loc[tru['true_id'] == c0, :] # True dets
    c0f = fal.loc[fal['true_id'] == c0, :] # False dets
    # Class 1 - 'illus'
    c1t = tru.loc[tru['true_id'] == c1, :]
    c1f = fal.loc[fal['true_id'] == c1, :]
    # Class 2 - 'map'
    c2t = tru.loc[tru['true_id'] == c2, :]
    c2f = fal.loc[fal['true_id'] == c2, :]
    # Class 3 - 'null'
    c3t = tru.loc[tru['true_id'] == c3, :]
    c3f = fal.loc[fal['true_id'] == c3, :]
    # Class 4 - 'phylo'
    c4t = tru.loc[tru['true_id'] == c4, :]
    c4f = fal.loc[fal['true_id'] == c4, :]


    # Plot parameters to make 1 subplot per image class
    kwargs = dict(alpha=0.5, bins=15)
    fig, axes = plt.subplots(len(imclasses), figsize=(10, 10), constrained_layout=True)
    fig.suptitle('Prediction Confidence by Class\n Overall Accuracy: {}'.format(
                  get_accuracy(len(tru), (len(tru)+len(fal)))))

    # Make subplots
    # Class 0 - 'herb'
    # True predictions
    axes[0].hist(c0t['confidence'], color='y', label='True Det', **kwargs)
    # False predictions
    axes[0].hist(c0f['confidence'], color='r', label='False Det', **kwargs)
    axes[0].set_title("{} (n={} images)\n Accuracy: {}".format(imclasses[0],
                      len(c0t+c0f), get_accuracy(len(c0t), (len(c0t)+len(c0f)))))
    axes[0].legend();

    # Class 1 - 'illus'
    # True predictions
    axes[1].hist(c1t['confidence'], color='y', label='True Det', **kwargs)
    # False predictions
    axes[1].hist(c1f['confidence'], color='r', label='False Det', **kwargs)
    axes[1].set_title("{} (n={} images)\n Accuracy: {}".format(imclasses[1],
                      len(c1t+c1f), get_accuracy(len(c1t), (len(c1t)+len(c1f)))))
    axes[1].legend();

    # Class 2 - 'map'
    # True predictions
    axes[2].hist(c2t['confidence'], color='y', label='True Det', **kwargs)
    # False predictions
    axes[2].hist(c2f['confidence'], color='r', label='False Det', **kwargs)
    axes[2].set_title("{} (n={} images)\n Accuracy: {}".format(imclasses[2],
                      len(c2t+c2f), get_accuracy(len(c2t), (len(c2t)+len(c2f)))))
    axes[2].legend();

    # Class 3 - 'null'
    # True predictions
    axes[3].hist(c3t['confidence'], color='y', label='True Det', **kwargs)
    # False predictions
    axes[3].hist(c3f['confidence'], color='r', label='False Det', **kwargs)
    axes[3].set_title("{} (n={} images)\n Accuracy: {}".format(imclasses[3],
                      len(c3t+c3f), get_accuracy(len(c3t), (len(c3t)+len(c3f)))))
    axes[3].legend();

    # Class 4 - 'phylo'
    # True predictions
    axes[4].hist(c4t['confidence'], color='y', label='True Det', **kwargs)
    # False predictions
    axes[4].hist(c4f['confidence'], color='r', label='False Det', **kwargs)
    axes[4].set_title("{} (n={} images)\n Accuracy: {}".format(imclasses[4],
                      len(c4t+c4f), get_accuracy(len(c4t), (len(c4t)+len(c4f)))))
    axes[4].legend();

    # Add Y-axis labels
    for ax in fig.get_axes():
        ax.set(ylabel='Freq (# imgs)')
        if thresh:
            ax.axvline(thresh, color='k', linestyle='dashed', linewidth=1)

    return fig

# To save a figure
def save_figure(fig, figtype, filetype='.png', taxon=None, TRAIN_SESS_NUM=TRAIN_SESS_NUM, handle_base=handle_base):
    # Make filename
    if taxon: # If for a specific taxon
        if 'plant' in taxon:
            handle_base = handle_base + '_plantae'
        elif 'anim' in taxon:
            handle_base = handle_base + '_animalia'

    outfpath = 'ratings_' + TRAIN_SESS_NUM + '_' + handle_base + '_' + figtype + filetype
    fig.savefig(outfpath, bbox_inches="tight")
    print("\nFigure(s) saved to: ", outfpath, "\n")

# To save a table
def save_table(tab, tabtype, filetype='.txt', TRAIN_SESS_NUM=TRAIN_SESS_NUM, handle_base=handle_base):
    # Make filename
    outfpath = 'ratings_' + TRAIN_SESS_NUM + '_' + handle_base + '_' + tabtype + filetype
    # Write table
    f = open(outfpath,"w+")
    f.writelines(tab)
    f.close()
    print("\nTable saved to: ", outfpath, "\n")

### Make classification report and plot confusion matrix of precision/recall for each image class
Use the confusion matrix and classification report table to evaluate model performance and determine class predictions to keep or filter out during post-processing. Precision and recall are related to accuracy. Precision measures what the model found (how many predicted positives are actually positive), while recall measures what the model missed (sensitivity / how many actual positives did the model identify).

$$precision = \frac{true\,positives}{true\,positives + false\,positives}$$

$$recall = \frac{true\,positives}{true\,positives + false\,negatives}$$

The classification report also includes an F1 score, which is the harmonic mean of precision and recall.

$$F1 = 2* \frac{precision*recall}{precision+recall}=\frac{2*true\,positives}{2*true\,positives+false\,positives+false\,negatives}$$

In [ ]:
#@title Make a classification report and confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
sns.set_style('darkgrid')

# Make and print classification report
df = all_predictions.replace('NaN', pd.NA).dropna(subset=['true_id', 'det_id'], ignore_index = True)
true_id = pd.DataFrame(df[['true_id']]).to_numpy()
det_id = pd.DataFrame(df[['det_id']]).to_numpy()
clr = classification_report(true_id, det_id, target_names = dataset_labels)
print("\nClassification report: \n", clr)

# Export classification report
save_table(clr, tabtype = 'classif_rep', filetype = '.txt')

# Make and plot confusion matrix
cm = confusion_matrix(true_id, det_id)
fig = plot_confusion_matrix(cm, dataset_labels, normalize=True)

# Export confusion matrix
save_figure(fig, figtype = 'conf_mx', filetype = '.png')

### Plot histograms of accuracy for each image class
Use these plots to determine confidence thresholds or class predictions to keep or filter out during post-processing.

In [ ]:
#@title Plot histograms (Optional: inspect for specific taxon and/or add a confidence threshold line)

# Optional: Inspect predictions for taxon-specific images only?
inspect_by_taxon = False #@param {type:"boolean"}
taxon = "" #@param {type:"string"}

# Optional: Draw threshold value to help choose optimal balance b/w maximizing useful data and minimizing error
thresh = 1.5 #@param {type:"number"}

# Valide predictions by image class (optionally, by taxon)
tru, fal, taxon = validate_predict(all_predictions, inspect_by_taxon, taxon)

# Plot result accuracy by image class (optionally, with confidence threshold line)
fig = plot_predict_x_conf(tru, fal, thresh, dataset_labels)

# Export histograms
save_figure(fig, figtype = 'hist', filetype = '.png', taxon = taxon)

### Simulate resulting dataset sizes based on different confidence thresholds

In [ ]:
# Load combined prediction results
df = all_predictions.copy()

# Split by True or False determined image ID
df['det'] = (df["true_id"] == df["det_id"])
tru = df.loc[df.det, :] # True ID
fal = df.loc[~df.det, :] # False ID

# Confidence values to test
conf_vals = [1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2] #@param
for conf_val in conf_vals:
    df_c = df.loc[df["confidence"] > conf_val, :]
    true_c = tru.loc[tru["confidence"] > conf_val, :]
    fal_c = fal.loc[fal["confidence"] > conf_val, :]
    all_vals = true_c.append(fal_c)
    print("\nConfidence Value: {}\n".format(conf_val))
    print("Accuracy for confidence > {}: {}".format(conf_val, get_accuracy(len(true_c), len(all_vals))))
    print("Predictions Retained (%): {}".format(len(df_c)/len(df)))
    print("True Predictions Retained (%): {}".format(format((len(true_c)/len(tru)), '.2f')))
    print("False Predictions Retained (%): {}".format(format((len(fal_c)/len(fal)), '.2f')))
    print("Accuracy for confidence > {}, by class:".format(conf_val))
    # By class
    for imclass in imclasses:
        true_det_c = len(true_c.loc[true_c["true_id"] == imclass, :])
        all_det_c = len(all_vals.loc[all_vals["true_id"] == imclass, :])
        accuracy = get_accuracy(true_det_c, all_det_c)
        print("{}: {}".format(imclass, accuracy))

## Inspect detections by image colorspace
---
Noticed that many false dets in illustrations were from greyscale color mode ('L' in pillow). Look at true and false detections for greyscale images in each class

In [ ]:
# Break up predictions by image class and colorspace

# Define variables
c0,c1,c2,c3,c4 = [imclasses[i] for i in range(0, len(imclasses))]
# Check how many true/false predictions are at each confidence value
# Class 0 - 'herb'
c0t = tru.loc[tru['true_id'] == c0, :] # True dets
c0f = fal.loc[fal['true_id'] == c0, :] # False dets
# Class 1 - 'illus'
c1t = tru.loc[tru['true_id'] == c1, :]
c1f = fal.loc[fal['true_id'] == c1, :]
# Class 2 - 'map'
c2t = tru.loc[tru['true_id'] == c2, :]
c2f = fal.loc[fal['true_id'] == c2, :]
# Class 3 - 'null'
c3t = tru.loc[tru['true_id'] == c3, :]
c3f = fal.loc[fal['true_id'] == c3, :]
# Class 4 - 'phylo'
c4t = tru.loc[tru['true_id'] == c4, :]
c4f = fal.loc[fal['true_id'] == c4, :]

# Class 0 - Herbarium Sheet
print("\n{}".format(c0))
print("False detections: {}\nTrue detections: {}".format(len(c0f), len(c0t)))
f_by_col = c0f.loc[c0f["colormode"]=="('L',)", :]
t_by_col = c0t.loc[c0t["colormode"]=="('L',)", :]
print("False for greyscale: {}\nTrue for greyscale: {}".format(len(f_by_col), len(t_by_col)))

# Class 1 - Illustration
print("\n{}".format(c1))
print("False detections: {}\nTrue detections: {}".format(len(c1f), len(c1t)))
f_by_col = c1f.loc[c1f["colormode"]=="('L',)", :]
t_by_col = c1t.loc[c1t["colormode"]=="('L',)", :]
print("False for greyscale: {}\nTrue for greyscale: {}".format(len(f_by_col), len(t_by_col)))

# Class 2 = Map
print("\n{}".format(c2))
print("False detections: {}\nTrue detections: {}".format(len(c2f), len(c2t)))
f_by_col = c2f.loc[c2f["colormode"]=="('L',)", :]
t_by_col = c2t.loc[c2t["colormode"]=="('L',)", :]
print("False for greyscale: {}\nTrue for greyscale: {}".format(len(f_by_col), len(t_by_col)))

# Class 3 = Null
print("\n{}".format(c3))
print("False detections: {}\nTrue detections: {}".format(len(c3f), len(c3t)))
f_by_col = c3f.loc[c3f["colormode"]=="('L',)", :]
t_by_col = c3t.loc[c3t["colormode"]=="('L',)", :]
print("False for greyscale: {}\nTrue for greyscale: {}".format(len(f_by_col), len(t_by_col)))

# Class 4 = Phylogeny
print("\n{}".format(c4))
print("False detections: {}\nTrue detections: {}".format(len(c4f), len(c4t)))
f_by_col = c4f.loc[c4f["colormode"]=="('L',)", :]
t_by_col = c4t.loc[c4t["colormode"]=="('L',)", :]
print("False for greyscale: {}\nTrue for greyscale: {}".format(len(f_by_col), len(t_by_col)))